# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [22]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [23]:
!cat talks.tsv

title	type	url_slug	venue	date	location	talk_url	description
How to Make the Most of a Postdoc	Talk	talk-15	American Historical Association (Moved online)	2020-10-08	Seattle, WA		
The Funding of Higher Education: Different Objectives in Switzerland, the UK, and the USA,	Roundtable	roundtable-2	Harvard University	2019-09-12	Cambridge, MA		
What is Research? Research Before, Alongside, and Outside the University	Talk	talk-15	Education for What? The History of Education and the Rise of the (American) Research University , Yale University	2019-02-22	New Haven, CT		
Loyalty/Disloyalty in Business	Roundtable	roundtable-1	Business History Conference Luncheon, AHA	2019-01-05	Chicago, IL		
Fixing genius, or how the romantic man of letters got a job	Talk	talk-14	Scientific persona and its incarnations—forms and functions of scientific and scholarly identity formation	2018-11-24	Stockholm, Sweden		
The Marketplace of Ideas: The Business of Knowing in Nineteenth-Century America	Talk	talk-13	Americ

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [24]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,How to Make the Most of a Postdoc,Talk,talk-15,American Historical Association (Moved online),2020-10-08,"Seattle, WA",NaN,NaN
1,The Funding of Higher Education: Different Obj...,Roundtable,roundtable-2,Harvard University,2019-09-12,"Cambridge, MA",NaN,NaN
2,"What is Research? Research Before, Alongside, ...",Talk,talk-15,Education for What? The History of Education a...,2019-02-22,"New Haven, CT",NaN,NaN
3,Loyalty/Disloyalty in Business,Roundtable,roundtable-1,"Business History Conference Luncheon, AHA",2019-01-05,"Chicago, IL",NaN,NaN
4,"Fixing genius, or how the romantic man of lett...",Talk,talk-14,Scientific persona and its incarnations—forms ...,2018-11-24,"Stockholm, Sweden",NaN,NaN
5,The Marketplace of Ideas: The Business of Know...,Talk,talk-13,American Historical Association,2018-01-04,"Washington, DC",NaN,NaN
6,Proprietors and Publics of Pacific History: Th...,Talk,talk-12,Making the Humanities VI,2017-09-29,"Oxford, UK",NaN,NaN
7,Selling Patronage: Toward a History of Subscri...,Talk,talk-11,"Society for the History of Authorship, Reading...",2017-06-10,"Victoria, BC",NaN,NaN
8,The Genius of History: Romantic Authorship and...,Talk,talk-10,The Persona of the Historian: Repertoires and ...,2017-01-27,"Leiden, NL",NaN,NaN
9,Beyond States of Mind: Denaturalizing the Nati...,Talk,talk-9,American Historical Association,2017-01-06,"Denver, CO",NaN,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [25]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [26]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [27]:
!ls ../_talks

2010-10-31-talk-1.md       2017-01-27-talk-10.md
2013-04-06-talk-2.md       2017-06-10-talk-11.md
2013-10-12-talk-3.md       2017-09-29-talk-12.md
2015-01-04-talk-4.md       2018-01-04-talk-13.md
2015-01-04-talk-5.md       2018-11-24-talk-14.md
2015-07-08-talk-6.md       2019-01-05-roundtable-1.md
2015-09-22-Roundtable-0.md 2019-02-22-talk-15.md
2015-11-19-talk-7.md       2019-09-12-roundtable-2.md
2016-07-04-talk-8.md       2020-10-08-talk-15.md
2017-01-06-talk-9.md


In [28]:
!cat ../_talks/2013-03-01-tutorial-1.md

cat: ../_talks/2013-03-01-tutorial-1.md: No such file or directory
